# ABS Inflation multi-measure

## Python set-up

In [1]:
# system imports
from typing import Callable, Sequence
from functools import cache

# analytic imports
import pandas as pd
from pandas import DataFrame
import numpy as np

# local imports
from readabs import (
    read_abs_series,
    qtly_to_monthly,
    percent_change,
    metacol,
    monthly_to_qtly,
    annualise_percentages,
)
from plotting import clear_chart_dir, line_plot, set_chart_dir, finalise_plot

In [2]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# save charts in this notebook
CHART_DIR = "./CHARTS/Inflation/"
set_chart_dir(CHART_DIR)
clear_chart_dir(CHART_DIR)

# some plotting constants
TARGET = {
    "ymin": 2,
    "ymax": 3,
    "color": "#dddddd",
    "label": "2-3% annual inflation target",
    "zorder": -1,
}
LFOOTER = "Australia. Orig = Original series. SA = Seasonally adjusted series. "

# display charts in this notebook
SHOW = False

## Main comparative charts

### Download key ABS data

In [3]:
@cache
def headline_capture() -> tuple[DataFrame, DataFrame, str]:
    """get key ABS inflation data."""

    # data we want to plot - these are through the year growth series
    mcpi_g = {
        "A128478318V": "Monthly CPI (Orig)",
        "A128481588C": "Monthly CPI (SA)",
        "A130184497K": "Monthly Trimmed Mean CPI (Orig)",
    }
    qcpi_g = {
        "A2325847F": "Qtly CPI (Orig)",
        "A3604508K": "Qtly CPI (SA)",
        "A3604511X": "Qtly Trimmed Mean CPI (SA)",
        "A3604505C": "Qtly Weighted Median CPI (SA)",
    }
    ppi_g = {
        "A2314867K": "Producer Price Index (Orig)",
    }
    wpi_g = {
        "A83895396W": "Wage Price Index (All sectors) (SA)",
    }
    gdp_g = {  # index numbers - NOT through the year % changes
        "A2303940R": "Household implicit price deflator (SA)",
        "A2303727C": "GNE implicit price deflator (SA)",
    }

    # data capture
    mcpi = read_abs_series("6484.0", mcpi_g, single_excel_only="648401")  # monthly CPI
    qcpi = read_abs_series(
        "6401.0", qcpi_g, single_excel_only="640106"
    )  # quarterly CPI
    ppi = read_abs_series(
        "6427.0", ppi_g, single_excel_only="642701"
    )  # producer price index
    wpi = read_abs_series(
        "6345.0", wpi_g, single_excel_only="634501"
    )  # wage price index`
    gdp = read_abs_series(
        "5206.0",
        gdp_g,
        single_excel_only="5206005_Expenditure_Implicit_Price_Deflators",
    )  # GDP implicit price deflators

    # calculate GDP implicit price deflator growth
    gdp = percent_change(gdp[0], 4), gdp[1]  # annualise

    # convert quarterly to mpnthly and merge
    data = mcpi[0]
    meta = mcpi[1]
    for d in qcpi, ppi, wpi, gdp:
        latest = qtly_to_monthly(d[0], interpolate=False)
        data = data.merge(latest, how="outer", left_index=True, right_index=True)
        meta = pd.concat([meta, d[1]])

    # rename columns
    for d in mcpi_g, qcpi_g, ppi_g, wpi_g, gdp_g:
        data = data.rename(columns=d)

    source = (
        f"ABS {', '.join(sorted(["6484.0", "6401.0", "6427.0", "6345.0", "5206.0"]))}"
    )
    return data, meta, source

### Multi-indicator plot

In [4]:
def plot_multi(
    starts: Sequence = ("1959-01-01", "2020-09-01", "2022-12-01"),
):
    """Plot multiple inflation indicators."""

    data, _meta, source = headline_capture()

    mixed_style = ["solid", "dotted", "dashed"] * 5
    marker_set = [
        "o",
        "v",
        "^",
        "<",
        ">",
        "8",
        "s",
        "p",
        "*",
        "h",
        "H",
        "D",
        "d",
        "P",
        "X",
    ]
    styles = (None, mixed_style, mixed_style)
    markers = (None, marker_set, marker_set)
    tags = ("", " (recent)", " (latest)")

    for start, marker, style, tag in zip(starts, markers, styles, tags):
        d = data.loc[start:]
        line_plot(
            d,
            title=f"Inflation Indicators{tag}",
            ylabel="Per cent (through the year)",
            axhspan=TARGET,
            y0=True,
            rfooter=source,
            legend={
                "loc": "best",
                "fontsize": "8",
                "ncol": 2,
            },
            lfooter=LFOOTER,
            show=SHOW,
            marker=marker,
            markersize=4,
            style=style,
            dropna=True,
        )


plot_multi()

### Quarterly/Monthly CPI pairs

In [5]:
def plot_pairs(
    function: Callable,
    pairs: dict[str, tuple[str, str]],
    start: str = "2019-12-01",
    title_stem: str = "Monthly vs Quarterly",
) -> None:
    """Plot pairs of inflation indicators."""

    data, _meta, source = function()
    data = data.loc[start:]
    for label, pair in pairs.items():
        d = data[list(pair)]
        line_plot(
            d,
            title=f"{label}: {title_stem}",
            ylabel="Per cent (through the year)",
            axhspan=TARGET,
            y0=True,
            rfooter=source,
            legend={"loc": "best", "fontsize": "8"},
            lfooter=LFOOTER,
            width=[1, 2],
            show=SHOW,
            dropna=True,
        )


headline_pairs = {
    # "label": ("quarterly series", "monthly series")
    "Headline CPI (Orig)": ("Qtly CPI (Orig)", "Monthly CPI (Orig)"),
    "Headline CPI (SA)": ("Qtly CPI (SA)", "Monthly CPI (SA)"),
    "Trimmed Mean CPI": (
        "Qtly Trimmed Mean CPI (SA)",
        "Monthly Trimmed Mean CPI (Orig)",
    ),
    "Weighted Median CPI": (
        "Qtly Weighted Median CPI (SA)",
        "Monthly Trimmed Mean CPI (Orig)",
    ),
}

plot_pairs(headline_capture, headline_pairs)

In [6]:
@cache
def analytic_capture() -> tuple[DataFrame, DataFrame, str]:
    """get key ABS analytic inflation data."""

    monthly = {  # through the year growth rates
        "A128483462J": "Monthly CPI Goods Component (Orig)",
        "A128481640A": "Monthly CPI Services Component (Orig)",
        "A128476506T": "Monthly CPI Tradables (Orig)",
        "A128480134T": "Monthly CPI Non-tradables (Orig)",
        "A130184498L": "Monthly CPI Excluding Volatile (SA)",
    }
    quarterly = {  # through the year growth rates
        "A2330617V": "Qtly CPI Goods Component (Orig)",
        "A2330707X": "Qtly CPI Services Component (Orig)",
        "A2330527R": "Qtly CPI Tradables (Orig)",
        "A2330572A": "Qtly CPI Non-tradables (Orig)",
        "A2330842R": "Qtly CPI Excluding Volatile (Orig)",
    }

    mon_data = read_abs_series("6484.0", monthly, single_excel_only="648401")
    qrt_data = read_abs_series("6401.0", quarterly, single_excel_only="640106")

    data2 = mon_data[0].merge(
        qtly_to_monthly(qrt_data[0], interpolate=False),
        how="outer",
        left_index=True,
        right_index=True,
    )
    for d in monthly, quarterly:
        data2 = data2.rename(columns=d)

    meta2 = pd.concat([mon_data[1], qrt_data[1]])
    text = ", ".join(sorted(meta2[metacol.cat].unique()))
    source2 = f"ABS: {text}"
    return data2, meta2, source2

In [7]:
analytic_pairs = {
    # "label": ("quarterly series", "monthly series")
    "CPI Goods Component": (
        "Qtly CPI Goods Component (Orig)",
        "Monthly CPI Goods Component (Orig)",
    ),
    "CPI Services": (
        "Qtly CPI Services Component (Orig)",
        "Monthly CPI Services Component (Orig)",
    ),
    "CPI Tradables": ("Qtly CPI Tradables (Orig)", "Monthly CPI Tradables (Orig)"),
    "CPI Non-tradables": (
        "Qtly CPI Non-tradables (Orig)",
        "Monthly CPI Non-tradables (Orig)",
    ),
    "CPI Excluding Volatile": (
        "Qtly CPI Excluding Volatile (Orig)",
        "Monthly CPI Excluding Volatile (SA)",
    ),
}

plot_pairs(analytic_capture, analytic_pairs)

In [8]:
# Lazy special case of quarterly goods vs services ...
lazy_pairs = {
    "CPI Goods and Services": (
        "Qtly CPI Goods Component (Orig)",
        "Qtly CPI Services Component (Orig)",
    ),
}

plot_pairs(analytic_capture, lazy_pairs, title_stem="Quarterly series")

## Annualised monthly/quarterly

***But note***: a whole host of problems annualising the monthly series.
And the data does not exist to annualise the monthly trimmed mean CPI.

In [9]:
def annualise_plot(
    data: tuple[DataFrame, DataFrame],
    pairs,
) -> None:
    """Plot annualised inflation indicators."""

    q_data, m_data = data
    for annualise in 3, 6:  # months
        # convert to six months growth
        qperiods, mperiods = int(annualise / 3), annualise
        q = percent_change(q_data, n_periods=qperiods)
        m = percent_change(m_data, n_periods=mperiods)

        # annualise
        q = annualise_percentages(q, 4 / qperiods)
        m = annualise_percentages(m, 12 / mperiods)

        # merge
        q = qtly_to_monthly(q, interpolate=False)
        merged = DataFrame(q).merge(m, how="outer", left_index=True, right_index=True)

        # plot
        def fake_capture(m=merged):
            return m, DataFrame(), "ABS: 6401.0, 6484.0"

        plot_pairs(
            fake_capture,
            pairs,
            title_stem=f"{annualise} months annualised",
        )


index_quarterly = {  # index numbers - NOT through the year % changes
    "A2325846C": "Qtly CPI (Orig)",
    "A3604506F": "Quartely CPI (SA)",
    "A2330841L": "Qtly CPI Excluding Volatile (Orig)",
    "A2330616T": "Qtly CPI Goods Component (Orig)",
    "A2330706W": "Qtly CPI Services Component (Orig)",
}
index_monthly = {  # index numbers - NOT through the year % changes
    "A128478317T": "Monthly CPI (Orig)",
    "A128481587A": "Monthly CPI (SA)",
    "A128473239F": "Monthly CPI Excluding Volatile (Orig)",
    "A128483461F": "Monthly CPI Goods Component (Orig)",
    "A128481639T": "Monthly CPI Services Component (Orig)",
}
index_pairs = {
    # "label": ("quarterly series", "monthly series")
    "Headline CPI": ("Qtly CPI (Orig)", "Monthly CPI (Orig)"),
    "Headline CPI (SA)": ("Quartely CPI (SA)", "Monthly CPI (SA)"),
    "CPI Excluding Volatile": (
        "Qtly CPI Excluding Volatile (Orig)",
        "Monthly CPI Excluding Volatile (Orig)",
    ),
    "CPI Goods Component": (
        "Qtly CPI Goods Component (Orig)",
        "Monthly CPI Goods Component (Orig)",
    ),
    "CPI Services Component": (
        "Qtly CPI Services Component (Orig)",
        "Monthly CPI Services Component (Orig)",
    ),
}

qindex = read_abs_series("6401.0", index_quarterly, single_excel_only="640106")[0]
mindex = read_abs_series("6484.0", index_monthly, single_excel_only="648401")[0]
qindex = qindex.rename(columns=index_quarterly)
mindex = mindex.rename(columns=index_monthly)


annualise_plot((qindex, mindex), index_pairs)

## Recent Phillips Curve

In [10]:
def phillips_curve() -> None:
    """Produce a Phillips Curve chart."""

    # trimmed mean annual inflation rate (seasonally adjusted)
    cpi, _ = read_abs_series("6401.0", "A3604509L", single_excel_only="640106")
    tm_cpi = cpi["A3604509L"].pct_change(periods=4, fill_method=None) * 100

    # seasonally adjusted unemployment rate
    lfs, _ = read_abs_series(
        "6202.0", ["A84423043C", "A84423047L"], single_excel_only="6202001"
    )
    ue_rate = monthly_to_qtly(100 - (lfs["A84423043C"] / lfs["A84423047L"] * 100))

    # select recent data
    start = "2021Q1"
    frame = pd.DataFrame({"Trimmed Mean CPI": tm_cpi, "_Unemployment Rate": ue_rate})
    frame = frame.loc[start:]

    # Fit a cubic regression line to the data
    model = np.poly1d(
        np.polyfit(frame["Trimmed Mean CPI"], frame["_Unemployment Rate"], 3)
    )
    polyline = np.linspace(
        frame["Trimmed Mean CPI"].min(), frame["Trimmed Mean CPI"].max(), 50
    )

    # plot
    ax = frame.plot(
        x="Trimmed Mean CPI",
        y="_Unemployment Rate",
        lw=2,
        label=f"Phillips curve since {start}",
    )
    ax.plot(
        polyline,
        model(polyline),
        color="darkred",
        linestyle="--",
        lw=0.75,
        label="Stylised Phillips curve (cubic regression)",
    )
    ax.axvline(
        2.5, color="darkorange", linestyle=":", lw=0.75, label="2.5% Inflation target"
    )
    for n in (0, -1):
        # Label the start and end
        ax.text(
            frame["Trimmed Mean CPI"].iloc[n],
            frame["_Unemployment Rate"].iloc[n],
            f"{frame.index[n]} ",
            fontsize="xx-small",
            ha="right",
        )
    finalise_plot(
        ax,
        title="Phillips Curve: Inflation vs Unemployment Rate",
        ylabel="Unemployment Rate (%)",
        xlabel="Trimmed Mean CPI Annual Growth Rate (%)",
        lfooter="Australia, Seasonally adjusted. Unemployment rate is quarterly mean. ",
        rfooter="Source: ABS 6202, 6401",
        legend={"fontsize": "x-small", "loc": "upper right"},
        show=SHOW,
    )


phillips_curve()

## Finished

In [11]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sat Jun 29 2024 06:22:28

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

numpy : 1.26.4
pandas: 2.2.2

Watermark: 2.4.3



In [12]:
print("Finished")

Finished
